# O método da Eliminação de Gauss em sistemas mal condicionados

Vamos implementar o método da Eliminação de Gauss sem qualquer preocupação com estratégia de pivoteamento. Aqui estamos interessados em avaliar como o mal condicionamento de uma matriz afeta a solução de um sistema linear.

In [1]:
using LinearAlgebra

Vamos considerar buscar a solção do sistema $Ax=b$. Para isso vamos implementar a função "eliminacao" que recebe $A$ e $b$ e retorna $\tilde{A}$ e $\tilde{b}$, onde $\tilde{A}$ é uma matriz triangular superior linha equivalente a $A$ e $\tilde{b}$ é linha equivalente a $b$.  

In [2]:
function eliminacao(A,b)
    n = size(b,1); 
    for k in 1 : n-1
        for i in k+1 : n
            m = A[i,k] / A[k,k];
            A[i,k] = 0.0;
            for j in k+1 : n
                A[i,j] = A[i,j] - m * A[k,j];
            end
            b[i] = b[i] - m * b[k];
        end
    end
    return(A,b)
end

eliminacao (generic function with 1 method)

A função "solucao" recebe as matrizes $\tilde{A}$ e $\tilde{b}$ da função anterior e calcular a solução do sistema por meio de uma substituição.

In [3]:
function solucao(A,b)
    n = size(b,1);
    x = zeros(n,1); 
    x[n] = b[n] / A[n,n];
    for k in n-1 : -1 : 1
        s = 0.0;
        for j in k+1 : n
            s = s + A[k,j] * x[j];
        end
        x[k] = (b[k] - s) / A[k,k];
    end
    return(x)
end
    

solucao (generic function with 1 method)

Com as funções acima, implementamos a função "eligauss" que dirigirá todo o processo de solução.

In [4]:
function eligauss(A,b)
    (A,b) = eliminacao(A,b);
    return solucao(A,b);
end

eligauss (generic function with 1 method)

## Exemplo

Vamos considerar a matriz de Hilbert, que é definida como $$H = \left[ \begin{array}{ccccc} 1 & \dfrac{1}{2} & \dfrac{1}{3} & \cdots & \dfrac{1}{n} \\  \\ \dfrac{1}{2} & \dfrac{1}{3} & \dfrac{1}{4} & \cdots & \dfrac{1}{n+1}  \\ \\ \vdots  & \vdots & \vdots & \vdots & \vdots  \\ \\  \dfrac{1}{n} & \dfrac{1}{n+1} & \dfrac{1}{n+2} & \cdots & \dfrac{1}{2n-1}\end{array}  \right].$$ Essa matriz é um exemplo clásico de matriz mal-condicionada. A função abaixo constrói essa matriz.

In [5]:
function hilbertmatrix(n)
    H = zeros(n,n);
    for i in 1:n
        for j in 1 : n
            H[i,j] = 1.0/(i+j-1);
        end
    end
    return H

end

hilbertmatrix (generic function with 1 method)

In [13]:
n = 20  # dimensão de A

A = hilbertmatrix(n)



20×20 Matrix{Float64}:
 1.0        0.5        0.333333   …  0.0555556  0.0526316  0.05
 0.5        0.333333   0.25          0.0526316  0.05       0.047619
 0.333333   0.25       0.2           0.05       0.047619   0.0454545
 0.25       0.2        0.166667      0.047619   0.0454545  0.0434783
 0.2        0.166667   0.142857      0.0454545  0.0434783  0.0416667
 0.166667   0.142857   0.125      …  0.0434783  0.0416667  0.04
 0.142857   0.125      0.111111      0.0416667  0.04       0.0384615
 0.125      0.111111   0.1           0.04       0.0384615  0.037037
 0.111111   0.1        0.0909091     0.0384615  0.037037   0.0357143
 0.1        0.0909091  0.0833333     0.037037   0.0357143  0.0344828
 0.0909091  0.0833333  0.0769231  …  0.0357143  0.0344828  0.0333333
 0.0833333  0.0769231  0.0714286     0.0344828  0.0333333  0.0322581
 0.0769231  0.0714286  0.0666667     0.0333333  0.0322581  0.03125
 0.0714286  0.0666667  0.0625        0.0322581  0.03125    0.030303
 0.0666667  0.0625     0.0

O número de condicionamento da matriz de hilbert é calculado abaixo.

In [14]:
cA = cond(A)

1.3193976166344822e18

Agora, vamos construir o sistema $$Ax=b$$ e fazer uma pequena perturbação no elemento $n$ do vetor $b$ e observar como essa pequena variação perturba fortemente a solução do sistema.

In [17]:
AA = copy(A);

x = ones(n,1);

b = A * x;

rb = copy(b)

# Introdução de um pequena perturbação em rb.
rb[n] = rb[n] - 1.e-10;

bb = copy(rb);

sol = eligauss(AA,bb)

20×1 Matrix{Float64}:
       1.0393988060753308
      -4.8994339325845
     217.07214585123356
   -3346.1172448505167
   26763.39065077833
 -118677.34470331525
  281402.6431752151
 -241652.086132807
 -348162.0277424745
  781872.765735821
  473877.6511906154
      -2.6226007397794193e6
       2.6899594036392863e6
      -1.0694461924004015e6
  483840.4332522352
  374484.47335137805
      -2.981880969277735e6
       4.1138435206361883e6
      -2.333537239308194e6
  493065.241137031

Abaixo calculamos a norma da diferença entre as soluções de $Ax=b$ e $Ax=\tilde{b}$. Além disso mostramos a norma da diferença entre $b$ e $\tilde{b}$.

In [16]:
ne = norm(x-sol);
nb = norm(b-rb);
println("Erro: $ne  ||b - ~b|| = $nb   N Cond = $cA")

Erro: 6.945883705108711e6  ||b - ~b|| = 1.000000082740371e-10   N Cond = 1.3193976166344822e18


Nota-se que uma pequena perturbação em $b$ introduziu uma grande perturbação na solução do problema. Isto se deve ao fato da matriz de Hilbert ser mal-condicionada.